<a href="https://colab.research.google.com/github/NULabTMN/hw5-aidasharif1365/blob/main/BeamSearch1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text generation with pre-trained Transformers
In this assignment we will work with Pre-trained Transformers such as GPT2 for generating text from a given sequence. Transformers aim to address the long term dependency issue in sequence-to-seuqence prediction by using concepts such as self-attention and positional encoding. GPT2 is a langauge model, pretrained on text generation, that can be used as a multi-task learner for tasks such as summarization, question-answering, and other generation tasks. This assignment's focus is on using GPT2 to generate text via greedy decoding and beam search. For more background on beam search, see [Jurafsky and Martin, chapter 11](https://web.stanford.edu/~jurafsky/slp3/11.pdf).

In [3]:
!pip install transformers

     |████████████████████████████████| 2.1MB 18.9MB/s 
     |████████████████████████████████| 3.3MB 52.3MB/s 
     |████████████████████████████████| 901kB 54.3MB/s 


In [4]:
import copy
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from scipy.special import softmax
import numpy as np

In [5]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [6]:
sentences = ['I like walking and', 
             'Martha wanted to read a book that',
             'Thomas is studying computer science to',
             'Their friendship inspired',
             'We should take the trash out since',
             'I am not a fan of coffee because',
             'I could not complete my homework by the deadline because',
             'The last semester was much easier due to',
             'I will be painting the walls white so that'
            ]

We apply greedy decoding to get predictions for each sentence here. This function returns the text output of greedy decoding. Modify it to return a tuple (ordered pair) of text and average log-likelihood per word for each sentence.

In [180]:
import math
import heapq
## TODO: Modify this function to return pairs of text and average log-likelihood
## per word for each sentence.
def greedy_decode(sentences, max_length, tokenizer):
  # Obtain loss from output and calculate
  # log likelihood for each sentence
  texts = []
  count=0
  for sentence in sentences:
    predicted_sentence = copy.copy(sentence)
    sum_logit=0
    # Predict a word each itertaion until the max length
    for i in range(max_length):
      indexed_tokens = tokenizer.encode(predicted_sentence)
      token_tensors = torch.tensor([indexed_tokens])

      with torch.no_grad():
        output = model(token_tensors, labels=token_tensors)
        predictions = output[1]
        sum_logit+=torch.max(predictions[0, -1, :]).item()
      predicted_index = torch.argmax(predictions[0, -1, :]).item()
      predicted_sentence = tokenizer.decode(indexed_tokens + [predicted_index])
    texts.append((predicted_sentence,sum_logit/(max_length)))
  return texts

In [181]:
texts = greedy_decode(sentences, max_length=25, tokenizer=tokenizer)
texts

[("I like walking and biking, but I don't like being in a car. I like to be in a car. I like to be in",
  -126.99030853271485),
 ('Martha wanted to read a book that she had read in college. She was a little nervous about it, but she was excited about it. She was a little',
  -121.15141571044921),
 ('Thomas is studying computer science to become a professor of computer science at the University of California, Berkeley. He is also a member of the Board of Trustees',
  -87.47114604949951),
 ("Their friendship inspired him to become a writer and a writer's assistant. He also wrote a book about his life and career.\n\n\n",
  -109.53943878173828),
 ('We should take the trash out since it\'s not going to be a problem," he said.\n\n\n"We\'re going to have to do something about',
  -102.8561047744751),
 ('I am not a fan of coffee because it is not good for you. I am a fan of coffee because it is not good for you.\n\n\nI',
  -91.627269115448),
 ('I could not complete my homework by the deadline 

You'll be implementing **beam search**, which returns a list of the $k$ most likely output sequences for each sentence. For this assignment, let $k = 8$. For the first token in the generated text, you will select the top $k$ output tokens. Then, for the next token, find the $k$-best continuations for each of those $k$ hypotheses and select the $k$-best overall. Return the $k$-best overall hypotheses according to average log likelihood per word. Note that if we don't average per word, the decoder will simply prefer shorter outputs. As above, return tuples of text output and avergae log likelihood.

In [201]:
#this function process only 1 sentence at a time
def beam_persentence(sentence, max_length, tokenizer, k):

  predicted_sentence = copy.copy(sentence)
  sents=[]
  indexed_tokens = tokenizer.encode(predicted_sentence)
  token_tensors = torch.tensor([indexed_tokens])
  with torch.no_grad():
    output = model(token_tensors, labels=token_tensors)
    predictions = output[1]
    sum_logit=torch.max(predictions[0, -1, :]).item()
  values,indices = torch.topk(predictions[0, -1, :],k)
  for ind in range(k):
    predicted_sentence = tokenizer.decode(indexed_tokens + [indices[ind].item()])
    sents.append((predicted_sentence,values[ind].item()))
    
  for i in range(max_length-1):
    sents2=[]
    for sentence in sents:
      predicted_sentence = copy.copy(sentence[0])
      indexed_tokens = tokenizer.encode(predicted_sentence)
      token_tensors = torch.tensor([indexed_tokens])
      with torch.no_grad():
        output = model(token_tensors, labels=token_tensors)
        predictions = output[1]
      #getting the top k values for likelihood and indices
      values,indices = torch.topk(predictions[0, -1, :],k)
      for ind in range(k):
        predicted_s = tokenizer.decode(indexed_tokens + [indices[ind].item()])
        #likelihood is divided by the number of words in a sentence
        sents2.append((predicted_s,(sentence[1]+values[ind].item())/predicted_s.count(' ')))
    #only getting k sentences with highest loglikelihood
    sents2=heapq.nlargest(k,sents2,key=lambda x:x[1])
    sents=sents2
  return sents2

In [202]:
## TODO: Implement beam search.
def beam_search(sentences, max_length, tokenizer, k):
  all_answers=[]
  for sentence in sentences:
    all_answers.append(beam_persentence(sentence, max_length, tokenizer, k))
  return all_answers
  

In [206]:
texts = beam_search(sentences, max_length=25, tokenizer=tokenizer,k=8)
texts

[[('I like walking and talking in front of the fireplace in front," said Dr Paul Denton of St John Hospital. Dr Dr Paul said there is',
   -2.1808825168629373),
  ('I like walking and talking in front of the fireplace in front," said Dr Paul Denton of St John Hospital. Dr Dr Paul said there are',
   -2.1838144931812966),
  ('I like walking and talking in front of the fireplace in front," said Dr Paul Denton of St John Hospital. Dr Dr Paul said there was',
   -2.1896998081227026),
  ('I like walking and talking in front of the fireplace in front," said Dr Paul Denton of St John Hospital. Dr Dr Paul said there were',
   -2.2078313694019998),
  ('I like walking and talking in front of the fireplace in front," said Dr Paul Denton of St John Hospital. Dr Dr Paul said there has',
   -2.2672959384937967),
  ('I like walking and talking in front of the fireplace in front," said Dr Paul Denton of St John Hospital. Dr Dr Paul said there have',
   -2.2705848178883277),
  ('I like walking and talk

In [204]:
texts = beam_search(sentences, max_length=25, tokenizer=tokenizer,k=2)
texts

[[('I like walking and I like playing with my friends," he says. "But it was just the way it worked out."<|endoftext|>The U.',
   1.5596639583003753),
  ('I like walking and I like playing with my friends," he says. "But it was just the way it worked out."<|endoftext|>The UES',
   1.5488080564914095)],
 [('Martha wanted to read a book that was about the history of the United States, and she wanted it to have an impact on her life, so that she could',
   -4.788773673061675),
  ('Martha wanted to read a book that was about the history of the United States, and she wanted it to have an impact on her life, so that she would',
   -4.855597172017763)],
 [('Thomas is studying computer science to become an assistant professor at Stanford University, where he is pursuing his PhD in computer science and computer engineering from the School for Information',
   -3.740836595146338),
  ('Thomas is studying computer science to become an assistant professor at Stanford University, where he is pursuin

In [205]:
texts = beam_search(sentences, max_length=25, tokenizer=tokenizer,k=1)
texts

[[("I like walking and biking, but I don't like being in a car. I like to be in a car. I like to be in",
   -5.619709010993636)],
 [('Martha wanted to read a book that she had read in college. She was a little nervous about it, but she was excited about it. She was a little',
   -4.036240599316462)],
 [('Thomas is studying computer science to become a professor of computer science at the University of California, Berkeley. He is also a member of the Board of Trustees',
   -3.184968301327893)],
 [("Their friendship inspired him to become a writer and a writer's assistant. He also wrote a book about his life and career.\n\n\n",
   -12.620372851760186)],
 [('We should take the trash out since it\'s not going to be a problem," he said.\n\n\n"We\'re going to have to do something about',
   -6.1592850726690065)],
 [('I am not a fan of coffee because it is not good for you. I am a fan of coffee because it is not good for you.\n\n\nI',
   -4.699249583776467)],
 [('I could not complete my homew

**TODO:** Record your observations here

The beam search is highly dependent on the log likelihood of previous sentences. If a specific path has lower log likelihood then the other possible sentences are discarded. In that sence the results for the simple word generator was better because it was taking into account the whole sentence not half sentences at each step.


For further exploration, you can experiment with $k$ to see how the fluency of text changes.

In my experiments lower k values resulted in better results.